<a href="https://colab.research.google.com/github/romitbarua/MultiModalDeepFake/blob/main/TestingNotebooks/tSNE_MFCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# t-SNE for MFCC

In [44]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import fnmatch
import os
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display
from sklearn.manifold import TSNE
import json
# Importing library 
import csv 
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import IPython.display as ipd
import plotly.express as px

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
filename = '/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb_v1.2/wav2lip_metadata.csv'
metadata_df = pd.read_csv(filename)

In [6]:
id76_df = metadata_df[metadata_df.source=='id00076']

In [9]:
id76_wav_filepaths = id76_df.wav_filepath.to_list()

In [34]:
def generate_scaled_mfcc_features(audio_file_path, plot=False):
  #generate audio time series
  audio,sample_rate = librosa.load(audio_file_path)

  #truncating feature for now 
  #may need to consider padding or interpolation
  #audio = audio[0:(3*sample_rate)]

  #replaced above with taking average across time

  #generate mfcc features
  mfcc_features = librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=20)
  #convert to double to avoid numeric computation issues
  mfcc_features = mfcc_features.astype(np.double)
  #scale the MFCCs such that each coefficient dimension has zero mean and unit variance
  scaled_mfcc_features = sklearn.preprocessing.scale(mfcc_features, axis=1)

  if plot==True:
    fig, ax = plt.subplots(nrows=1, figsize=(15, 7), sharex=True, sharey=True)
    mfcc_plot = display.specshow(mfcc_features, sr=sample_rate, x_axis='time', ax=ax)
    ax.set(title='MFCC spectrogram for {}'.format(audio_file_path.split('/')[-2]))
    fig.colorbar(mfcc_plot, ax=ax)
    return scaled_mfcc_features, mfcc_plot
  
  return scaled_mfcc_features

In [39]:
feature_vectors = []
avg_mfccs = []
#iterate through all wav files for id
for wav_file in id76_wav_filepaths:
  #generate mfcc
  mfccs = generate_scaled_mfcc_features(wav_file)
  #full mfcc feature vector without padding or truncation
  feature_vectors.append(mfccs)
  #average mfcc feature vector
  avg_mfccs.append(np.mean(mfccs,1))

In [40]:
len(feature_vectors), feature_vectors[0].shape

(21, (20, 435))

In [41]:
#averaging for tSNE
for vector in feature_vectors:
  print(vector.shape)
  avg_mfcc = np.mean(vector,1)
  print(avg_mfcc.shape)

(20, 435)
(20,)
(20, 280)
(20,)
(20, 392)
(20,)
(20, 340)
(20,)
(20, 202)
(20,)
(20, 183)
(20,)
(20, 247)
(20,)
(20, 199)
(20,)
(20, 190)
(20,)
(20, 221)
(20,)
(20, 280)
(20,)
(20, 435)
(20,)
(20, 359)
(20,)
(20, 314)
(20,)
(20, 321)
(20,)
(20, 185)
(20,)
(20, 223)
(20,)
(20, 354)
(20,)
(20, 255)
(20,)
(20, 221)
(20,)
(20, 158)
(20,)


In [42]:
len(avg_mfccs), avg_mfccs[0].shape

(21, (20,))

In [43]:
model = TSNE(n_components=2, 
             learning_rate=200, #default value 
             perplexity=5, #must be lower than num samples
             verbose=2).fit_transform(avg_mfccs)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,


[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 21 samples in 0.000s...
[t-SNE] Computed neighbors for 21 samples in 0.003s...
[t-SNE] Computed conditional probabilities for sample 21 / 21
[t-SNE] Mean sigma: 0.000000
[t-SNE] Computed conditional probabilities in 0.010s
[t-SNE] Iteration 50: error = 45.4027634, gradient norm = 0.4395739 (50 iterations in 0.020s)
[t-SNE] Iteration 100: error = 50.8502464, gradient norm = 0.3700581 (50 iterations in 0.017s)
[t-SNE] Iteration 150: error = 47.4991875, gradient norm = 0.5061514 (50 iterations in 0.015s)
[t-SNE] Iteration 200: error = 44.1463051, gradient norm = 0.4709113 (50 iterations in 0.018s)
[t-SNE] Iteration 250: error = 44.5103683, gradient norm = 0.4816028 (50 iterations in 0.017s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 44.510368
[t-SNE] Iteration 300: error = 0.5664833, gradient norm = 0.0009703 (50 iterations in 0.015s)
[t-SNE] Iteration 350: error = 0.5042081, gradient norm = 0.0003030 (50 i

In [45]:
x_axis=model[:,0]
y_axis=model[:,1]
fig = px.scatter(x=x_axis, y=y_axis)
fig.show()

## For multiple IDs


In [49]:
metadata_df.columns

Index(['Unnamed: 0', 'source', 'target1', 'target2', 'method', 'category',
       'type', 'race', 'gender', 'filename', 'directory_path', 'mp4_filepath',
       'utterance_file', 'wav_filepath'],
      dtype='object')

In [67]:
all_uniq_ids = metadata_df.source.unique()
#select 20
analysis_ids = all_uniq_ids[:20]
#list for filepaths
wav_filepaths = []
sources = []
color = []
targets = []
category = []
#get data
col = 1
for id in analysis_ids:
  temp_df = metadata_df[metadata_df.source==id]
  wav_filepaths.extend(temp_df.wav_filepath.to_list())
  sources.extend(temp_df.source.to_list())
  targets.extend(temp_df.target1.to_list())
  category.extend(temp_df.category.to_list())
  color.extend(len(temp_df.source.to_list())*[col])
  col += 1

In [65]:
len(wav_filepaths), len(sources), len(targets), len(category), len(color)

(427, 427, 427, 427, 427)

In [57]:
feature_vectors = []
avg_mfccs = []
#iterate through all wav files for id
for wav_file in wav_filepaths:
  #generate mfcc
  mfccs = generate_scaled_mfcc_features(wav_file)
  #full mfcc feature vector without padding or truncation
  feature_vectors.append(mfccs)
  #average mfcc feature vector
  avg_mfccs.append(np.mean(mfccs,1))

In [58]:
model = TSNE(n_components=2, 
             learning_rate=200, #default value 
             perplexity=5, #must be lower than num samples
             verbose=2).fit_transform(avg_mfccs)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.



[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 427 samples in 0.000s...
[t-SNE] Computed neighbors for 427 samples in 0.011s...
[t-SNE] Computed conditional probabilities for sample 427 / 427
[t-SNE] Mean sigma: 0.000000
[t-SNE] Computed conditional probabilities in 0.024s
[t-SNE] Iteration 50: error = 68.7777557, gradient norm = 0.5087144 (50 iterations in 0.135s)
[t-SNE] Iteration 100: error = 70.9905701, gradient norm = 0.4987022 (50 iterations in 0.055s)
[t-SNE] Iteration 150: error = 72.3117447, gradient norm = 0.4812205 (50 iterations in 0.058s)
[t-SNE] Iteration 200: error = 73.7718658, gradient norm = 0.4528584 (50 iterations in 0.054s)
[t-SNE] Iteration 250: error = 70.3758087, gradient norm = 0.4980394 (50 iterations in 0.051s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 70.375809
[t-SNE] Iteration 300: error = 1.1359257, gradient norm = 0.0022249 (50 iterations in 0.052s)
[t-SNE] Iteration 350: error = 1.1298730, gradient norm = 0.0002336 (

In [70]:
x_axis=model[:,0]
y_axis=model[:,1]
fig = px.scatter(x=x_axis, 
                 y=y_axis,
                 symbol=category,
                 color=sources)
fig.show()